# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
import os
from tqdm import tqdm
import glob
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import plotly.express as px
import statsmodels.formula.api as smf
import plotly.graph_objects as go
from statsmodels.stats.multitest import multipletests
import plotly.io as pio
pio.kaleido.scope.mathjax = None
from plotly.offline import init_notebook_mode
from matplotlib import patheffects as pe
import matplotlib
import warnings
init_notebook_mode(connected=False)
from scipy.stats import mannwhitneyu, median_test, kruskal, wilcoxon, friedmanchisquare
import matplotlib.pyplot as plt
import pathlib
from sklearn.metrics import mean_absolute_error
from statannotations.Annotator import Annotator
import functools
import matplotlib.lines as mlines
import patchworklib as pw
import pickle
from src.routines.plotly_layout import add_layout, color_tick
from d3blocks import D3Blocks

# Collect data

In [ ]:
path = "D:/YandexDisk/Work/pydnameth/draft/10_MetaEPIClock/MetaEpiAge"
df = pd.read_excel(f"{path}/table.xlsx", index_col=0)
colors_gpls = {
    'GPL13534': 'firebrick',
    'GPL16304': 'orangered',
    'GPL21145': 'darkorchid',
    'GPL23976': 'orchid'
}
df_gpls = {}
for gpl in colors_gpls:
    df_gpls[gpl] = pd.read_excel(f"{path}/figures/gpls/{gpl}.xlsx", index_col=0)
    
colors_xkcd = matplotlib.colors.XKCD_COLORS
df_colors = pd.read_excel(f"{path}/colors.xlsx", index_col=0)
colors_global = {}
for feat in df_colors.index.values:
    colors_global[feat] = colors_xkcd[f"xkcd:{df_colors.at[feat, 'xkcd']}"]

# Plot samples count hist

In [ ]:
path_save = f"{path}/figures/gpls/histplot"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)

hist_bins = np.linspace(0, 4000, 41)

df_stat = pd.DataFrame(index=list(colors_gpls.keys()), columns=['Total', 'Processed', 'Processed %', 'Selected', 'Selected %'])
for gpl, color in colors_gpls.items():
    fig, ax = plt.subplots(figsize=(4, 3))
    sns.set_theme(style='ticks')
    histplot = sns.histplot(
        data=df_gpls[gpl],
        x=f"Count",
        bins=hist_bins,
        edgecolor='silver',
        linewidth=0.5,
        color=color,
        ax=ax
    )
    ax.set_xlabel('Number of samples in GSE')
    ax.set_ylabel('Number of GSEs')
    ax.set_title(gpl)
    ax.set_xlim([-15, 4000])
    ax.patches[0].set_facecolor('gainsboro')
    for container in ax.containers:
        ax.bar_label(container, fmt='%d', fontsize=5, color='deepskyblue')
    sns.despine(left=False, right=True, bottom=False, top=True)
    plt.savefig(f"{path_save}/{gpl}.png", bbox_inches='tight', dpi=400)
    plt.savefig(f"{path_save}/{gpl}.pdf", bbox_inches='tight')
    plt.close(fig)
    
    count_total = df_gpls[gpl]['Count'].sum()
    count_proc = df_gpls[gpl].loc[df_gpls[gpl]['Count'] >= 100, 'Count'].sum()
    count_slctd = len(df.index[df['GPL']==gpl])
    df_stat.at[gpl, 'Total'] = count_total
    df_stat.at[gpl, 'Processed'] = count_proc
    df_stat.at[gpl, 'Processed % '] = count_proc / count_total * 100
    df_stat.at[gpl, 'Selected'] = count_slctd
    df_stat.at[gpl, 'Selected % '] = count_slctd / count_total * 100

df_stat.to_excel(f"{path_save}/stat.xlsx", index_label="GPL")

# Plot sunburst

In [ ]:
path_save = f"{path}/figures/gpls/sunburst"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)

for gpl, color in colors_gpls.items():
    labels = ['Processed', 'Not processed', 'Selected', '']
    texts = ['Processed', 'Not processed', 'Selected', '']
    parents = ['', '', 'Processed', 'Processed']
    values = [
        df_gpls[gpl].loc[df_gpls[gpl]['Count'] >= 100, 'Count'].sum(),
        df_gpls[gpl].loc[df_gpls[gpl]['Count'] < 100, 'Count'].sum(),
        len(df.index[df['GPL']==gpl]),
        df_gpls[gpl].loc[df_gpls[gpl]['Count'] >= 100, 'Count'].sum() - len(df.index[df['GPL']==gpl])
    ]
    colors = [color, 'gainsboro', color, 'white']
    
    df_gpl = df.loc[df['GPL']==gpl, :]
    df_states = df_gpl['State'].value_counts().to_frame()
    for state in df_states.index:
        labels.append(state)
        texts.append('')
        parents.append('Selected')
        values.append(df_states.at[state, 'State'])
        colors.append(colors_global[state])
    
    fig = go.Figure()

    fig.add_trace(
        go.Sunburst(
            labels=labels,
            parents=parents,
            values=values,
            text=texts,
            branchvalues="total",
            textinfo = "text",
            # textfont_size=10,
            marker=dict(
                colors=colors,
                line=dict(color='black', width=0.5)
                # pattern=dict(
                #     shape=["", "/", "/", ".", ".", "/", "/", ".", "/"], solidity=0.9
                # )
            ),
        )
    )
    add_layout(fig, f"", f"", f"")
    fig.update_layout(margin=dict(t=0, l=0, r=0, b=0))
    fig.write_image(f"{path_save}/{gpl}.png", scale=2)
    fig.write_image(f"{path_save}/{gpl}.pdf", format="pdf")

    hist_bins = np.linspace(0, 110, 12)
    fig, ax = plt.subplots()
    sns.set_theme(style='whitegrid')
    histplot = sns.histplot(
        data=df_gpl,
        x=f"Age",
        bins=hist_bins,
        edgecolor='k',
        linewidth=0.5,
        hue_order=list(df_states.index),
        hue="State",
        palette=colors_global,
        multiple="stack",
        ax=ax
    )
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
    plt.savefig(f"{path_save}/{gpl}_states.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path_save}/{gpl}_states.pdf", bbox_inches='tight')
    plt.close(fig)

# Plot reference dataset distribution

In [ ]:
path_save = f"{path}/figures/ref"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)

df_ref = df.loc[df['GSE']=='GSE87571', :]
hist_bins = np.linspace(0, 110, 23)
fig, ax = plt.subplots(figsize=(5, 3))
sns.set_theme(style='ticks')
histplot = sns.histplot(
    data=df_ref,
    x=f"Age",
    bins=hist_bins,
    edgecolor='k',
    linewidth=0.5,
    color='cornflowerblue',
    ax=ax
)
ax.set_title('GSE87571')
plt.savefig(f"{path_save}/hist_age.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/hist_age.pdf", bbox_inches='tight')
plt.close(fig)